# Solver configuration as a bandit problem

In this tutorial we are interested in minimizing the expected branch-and-bound tree size of the SCIP solver by tuning the parameter [`branching/scorefac`](https://www.scipopt.org/doc/html/PARAMETERS.php), which takes values in the range $[0,1]$. This parameter, used in combination with the sum score function (`branching/scorefunc=s`), controls the weighting of downward and upward gain predictions in the computation of branching scores. It has a default value of 0.167.

Dependencies are given for conda in the file `conda-requirements.yaml`

Train, optimization, and test parameters of the notebook.

In [ ]:
# Parameters set Papermill during testing, do not rename.
train_n_items = 100
train_n_bids = 100
train_add_item_prob = 0.7
optim_n_iters = 100
optim_n_burnins = 10
optim_seed = 42
test_n_items = 150
test_n_bids = 750
test_add_item_prob = 0.7
test_seed = 1337
test_n_evals = 5

In [ ]:
import ecole as ec
import matplotlib.pyplot as plt
import numpy as np
import skopt


## 1. Setting up the Ecole environment

We formulate this parameter tuning task as a continuum-armed bandit problem, which we instantiate using a [`Configuring`](https://doc.ecole.ai/master/reference/environments.html#configuring) environment. We request no observation (non-contextual bandit), and use the negative number of nodes as a reward (tree size minimization).

In [ ]:
env = ec.environment.Configuring(
    # set up a few SCIP parameters
    scip_params={
        "branching/scorefunc": "s",  # sum score function
        "branching/vanillafullstrong/priority": 666666,  # use vanillafullstrong (highest priority)
        "presolving/maxrounds": 0,  # deactivate presolving
    },
    # pure bandit, no observation
    observation_function=None,
    # minimize the total number of nodes
    reward_function=-ec.reward.NNodes(),
    # collect additional metrics for information purposes
    information_function={
        "nnodes": ec.reward.NNodes().cumsum(),
        "lpiters": ec.reward.LpIterations().cumsum(),
        "time": ec.reward.SolvingTime().cumsum(),
    },
)

We set up SCIP to use the sum score function for branching (`branching/scorefunc=s`), and the *vanillafullstrong* branching rule to mitigate the impact of branching heuristics (`branching/vanillafullstrong/priority=666666`). For the purpose of the tutorial we also deactivate presolving (`presolving/maxrounds=0`) in order to reduce computational time.

## 2. Setting up the training distribution

For the purpose of this tutorial we will consider randomly generated Combinatorial Auction problems, as the problem distribution for which we want to configure the solver. We hence set up a `CombinatorialAuctionGenerator` that will generate such instances on the fly.

In [ ]:
# infinite instance generator, new instances will be generated on-the-fly
instances = ec.instance.CombinatorialAuctionGenerator(
    n_items=train_n_items, n_bids=train_n_bids, add_item_prob=train_add_item_prob
)

For training we consider small-sized instances ($100\times 100$), which are solved within seconds by SCIP but are difficult enough to produce tens of branch-and-bound nodes.

## 3. Solving the control problem

We can now readily solve the optimization problem using an off-the-shelf optimization library, such as `scikit-optimize`.

In [ ]:
env.seed(optim_seed)  # environment (SCIP)
instances.seed(optim_seed)  # instance generator
rng = np.random.RandomState(optim_seed)  # optimizer

# set up the optimizer
opt = skopt.Optimizer(
    dimensions=[(0.0, 1.0)],
    base_estimator="GP",
    n_initial_points=optim_n_burnins,
    random_state=rng,
    acq_func="PI",
    acq_optimizer="sampling",
    acq_optimizer_kwargs={"n_points": 10},
)

assert optim_n_iters > optim_n_burnins

# run the optimization
for i in range(optim_n_iters):

    if (i + 1) % 10 == 0:
        print(f"iteration {i+1} / {optim_n_iters}")

    # pick up a new random instance
    instance = next(instances)

    # start a new episode
    env.reset(instance)

    # get the next action from the optimizer
    x = opt.ask()
    action = {"branching/scorefac": x[0]}

    # apply the action and collect the reward
    _, _, reward, _, _ = env.step(action)

    # update the optimizer
    opt.tell(x, -reward)  # minimize the negated reward (eq. maximize the reward)

We can now visualize the result of the optimization process.

In [ ]:
model = opt.models[-1]

x = np.linspace(0, 1, 500)
x_model = opt.space.transform(x.reshape(-1, 1).tolist())

fig, ax1 = plt.subplots()

# points sampled during optimization
lns1 = ax1.plot(opt.Xi, opt.yi, "r.", markersize=8, label="Collected data")

# value function estimation
y_mean, y_std = model.predict(x_model, return_std=True)
lns2 = ax1.plot(x, y_mean, "g--", label=r"Value function")
ax1.fill_between(
    x, y_mean - 1.6 * y_std, y_mean + 1.6 * y_std, alpha=0.2, fc="g", ec="None"
)

# probability of improvement estimation
x_pi = skopt.acquisition.gaussian_pi(x_model, model, y_opt=np.min(opt.yi))
ax2 = ax1.twinx()
lns3 = ax2.plot(x, x_pi, "b", label="Prob. of improvement")

ax1.set_title(f"Model obtained after {optim_n_iters} iterations")
ax1.set_ylabel(f"number of nodes (neg. reward)")
ax1.set_xlabel(f"$branching/scorefac$ parameter value (action)")

ax2.set_ylabel(f"Probability value")

# Legend
lns = lns1 + lns2 + lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc="upper center")

plt.show()

# get best value based on a grid search on the value function estimator
best_value = x[np.argmin(y_mean)]
print(f"Best parameter value: branching/scorefac = {best_value}")

## 4. Evaluation on harder instances

In [ ]:
# we set up more challenging instances
test_instances = ec.instance.CombinatorialAuctionGenerator(
    n_items=test_n_items, n_bids=test_n_bids, add_item_prob=test_add_item_prob
)

for policy in ("default", "learned"):

    print(f"evaluating policy '{policy}'")
    results = []

    for i in range(test_n_evals):

        # evaluate each policy in the exact same settings
        env.seed(test_seed + i)  # environment (SCIP)
        test_instances.seed(test_seed + i)  # instance generator

        # pick up the next instance
        instance = next(test_instances)

        # set up the episode initial state
        env.reset(instance)

        # get the action from the policy
        if policy == "default":
            action = {}  # will use the default value from SCIP
        else:
            action = {"branching/scorefac": best_value}

        # apply the action and collect the reward
        _, _, _, _, info = env.step(action)

        print(
            f"  instance {i+1}: {info['nnodes']} nodes, {info['lpiters']} lpiters, {info['time']} secs"
        )

        results.append(info["nnodes"])

    print(f"  average performance: {np.mean(results)} nodes")